In [1]:
import torch
import numpy as np
from torch_geometric.loader import DataLoader

# from Models.GNN import IGCNet
from Utils.data_gen import Generate_Input, create_graph
from Utils.training import sr_loss_matrix, average_weights
%load_ext autoreload
%autoreload 2


# Parameters

In [2]:
num_H = 8
num_test = 20
K = 5 # number of terminals
M = 10 # number of access points
B = 20 # Mhz
D = 1 # km
tau=10
random_matrix = np.random.randn(tau, tau)
U, S, V = np.linalg.svd(random_matrix)

Hb = 15 # Base station height in m
Hm = 1.65 # Mobile height in m
f = 1900 # Frequency in MHz
aL = (1.1 * np.log10(f) - 0.7) * Hm - (1.56 * np.log10(f) - 0.8)
L = 46.3+33.9*np.log10(f)-13.82*np.log10(Hb)-aL

power_f=0.2 # downlink power
# Pd = power_f / 10 ** ((-203.975 + 10 * np.log10(20 * 10 ** 6) + 9) / 10) # normalized receive SNR
Ther_noise = 20000000 * 10**(-17.4) * 10**-3
Pd = 1/Ther_noise
Pu=Pd

d0=0.01 # km
d1=0.05 # km

N = 50

R_cf_min = np.zeros(N)  # Cell Free
R_cf_sum = np.zeros(N)
R_cf_opt_min = np.zeros(N)

In [3]:
num_train = 4
num_test = 2
batchSize = 32
num_rounds = 20

num_epochs = 500
lr = 1e-4
step_size = 5
gamma = 0.9

In [19]:
num_AP = M 
Beta_all, Phi_all = Generate_Input(num_train, tau, K, M, Pd, D=1, Hb=15, Hm=1.65, f=1900,
                    var_noise=1, Pmin=0, seed=2017, d0=d0, d1=d1)
train_data = create_graph(Beta_all, Phi_all)
train_loader = [
    DataLoader(train_data[i], batch_size=batchSize, shuffle=True)
    for i in range(num_AP)
]

Beta_test, Phi_test = Generate_Input(num_test, tau, K, M, Pd, D=1, Hb=15, Hm=1.65, f=1900,
                    var_noise=1, Pmin=0, seed=2017, d0=d0, d1=d1)
test_data = create_graph(Beta_test, Phi_test)
test_loader = [
    DataLoader(test_data[i], batch_size=batchSize, shuffle=False)
    for i in range(num_AP)
]

In [ ]:

def per_AP_calculate(power, data):
    return None, None, None

def data_process(ds, pc, ui):
    return None

def return_graph(data):
    return None

In [23]:
for batches in zip(*train_loader): 
    print(batches[9])
    # break

DataBatch(x=[20, 12], edge_index=[2, 80], edge_attr=[80, 2], batch=[20], ptr=[5])


In [ ]:
from Models.GNN import APNet
from Utils.training import rate_calculation

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

global_model = APNet(node_dim=12, edge_dim=2, hidden_dim=32).to(device)
local_models, optimizers = [], []

# Init every client model/optimizer
for each_AP in range(num_AP):
    model = APNet(node_dim=12, edge_dim=2, hidden_dim=32).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    local_models.append(model)
    optimizers.append(optimizer)

for round in range(num_rounds):
    # zip loaders to align batches across APs
    for batches in zip(*train_loader):                       # sync step across APs
        per_ap_cache, send_to_server = [], []
        for model, opt, batch in zip(local_models, optimizers, batches):
            model.eval()
            batch = batch.to(device)

            power = model(batch)                             # your APNet output
            ds_k, pc_k, ui_k = per_AP_calculate(power, batch)  # implement physics here
            send_to_server.append({"ds": ds_k.detach(),
                            "pc": pc_k.detach(),
                            "ui": ui_k.detach()})
            per_ap_cache.append((ds_k, pc_k, ui_k, opt))

        return_from_server = return_graph(send_to_server)
        # Training each client with the coresponding data/model/compliment
        for model, opt, batch, compliment in zip(local_models, optimizers, batches, return_from_server):
            model.train(); opt.zero_grad(); batch = batch.to(device)
            local_loss = train(batch, model, opt, compliment)
            local_loss.backward()
            opt.step()

    # --- FedAvg (optional, by sample count) ---
    local_weights = [m.state_dict() for m in local_models]
    sizes = [len(dl.dataset) for dl in train_loader]
    new_state = average_weights(local_weights, sizes)
    global_model.load_state_dict(new_state)
    
    # Update client models
    for m in local_models: m.load_state_dict(global_model.state_dict())
    



SyntaxError: incomplete input (1510043625.py, line 35)

In [6]:
from Utils.training import train, eval

num_rounds = 1
for round in range(num_rounds):
    local_weights = []
    
    # Local training
    for i in range(num_AP):
        dataLoader, model, optimizer = train_loader[i], local_models[i], optimizers[i]
        break
        local_loss = train(train_loader[i], local_models[i], optimizers[i])
        local_weights.append(local_models[i].state_dict())

    # Server aggregation (FedAvg)
    # new_state = average_weights(local_weights)
    # global_model.load_state_dict(new_state)

    # # Distribute updated global model back to all clients
    # for i in range(num_AP):
    #     local_models[i].load_state_dict(global_model.state_dict())

In [283]:
for batch in dataLoader:
    power = model(batch)
    break

# Loss function

In [295]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
Ther_noise = 20000000 * 10**(-17.4) * 10**-3
Pp = 1/Ther_noise
NoAntennas = 10


num_graphs = batch.num_graphs
node_feat = batch.x
edge_attr = batch.edge_attr

num_UE = node_feat.shape[0]//num_graphs

node_matrix = batch.x.reshape(num_graphs, num_UE, -1)
power = power.reshape(num_graphs, num_UE, 1)

direct_channel = node_matrix[:,:,:1]
pilot_assignment = node_matrix[:,:,1:-1]

# Desire Signal - Fixed for each AP - no need exchange
desired_signal = direct_channel * power

# Pilot Contamination - Fixed for each AP
pilot_contamination = torch.bmm(
    pilot_assignment,                     # (num_graphs, num_UE, tau)
    pilot_assignment.transpose(1, 2),      # (num_graphs, tau, num_UE)
).abs()

print(f'node_feat: {node_feat.shape}')
print(f'node_feat: {direct_channel.shape}')

print(f'edge_attr: {edge_attr.shape}')

node_feat: torch.Size([20, 12])
node_feat: torch.Size([4, 5, 1])
edge_attr: torch.Size([80, 2])


In [296]:
print('Local data of each AP:')
print('Node feature:')
print(f'\t desired large scale + tau + hidden dim: {node_matrix.shape}')
print('Edge attribute:')
print(f'\t large scale fading (2 ways): {batch.edge_attr.shape}')

print(f'Large scale fading: \t {direct_channel.shape}')
print(f'Power of each AP: \t {power.shape}')
print(f'Pilot: \t\t\t {pilot_assignment.shape}')


Local data of each AP:
Node feature:
	 desired large scale + tau + hidden dim: torch.Size([4, 5, 12])
Edge attribute:
	 large scale fading (2 ways): torch.Size([80, 2])
Large scale fading: 	 torch.Size([4, 5, 1])
Power of each AP: 	 torch.Size([4, 5, 1])
Pilot: 			 torch.Size([4, 5, 10])


In [292]:
largeScale = torch.rand(num_graphs, num_AP, num_UE, device=device) # beta
channelVariance = torch.rand(num_graphs, num_AP, num_UE, device=device)
powerMatrix = torch.rand(num_graphs, num_AP, num_UE, device=device) # sqrt of the power
pilotAssignment = torch.rand(num_graphs, num_UE, tau, device=device)


In [293]:
print("Requirement to calculate rate:")
print(f'\t Full large scale fading: \t {largeScale.shape}')
print(f'\t Power allocation: \t\t {powerMatrix.shape}')
print(f'\t Pilot: \t\t\t {pilotAssignment.shape}')

Requirement to calculate rate:
	 Full large scale fading: 	 torch.Size([4, 10, 5])
	 Power allocation: 		 torch.Size([4, 10, 5])
	 Pilot: 			 torch.Size([4, 5, 10])


## Test

In [226]:
largeScale = torch.rand(num_graphs, num_AP, num_UE, device=device) # beta
channelVariance = torch.rand(num_graphs, num_AP, num_UE, device=device)
powerMatrix = torch.rand(num_graphs, num_AP, num_UE, device=device) # sqrt of the power
pilotAssignment = torch.rand(num_graphs, num_UE, tau, device=device)

SINR_num = torch.sum(powerMatrix*channelVariance, dim=1) ** 2

powerExpanded = (powerMatrix*channelVariance).unsqueeze(-1)
largeScaleExpanded = largeScale.unsqueeze(-2)
userInterference = torch.sum(powerExpanded * largeScaleExpanded, dim=(1, 2))

interm_var1 = (powerMatrix/ largeScale).unsqueeze(-1)
interm_var2 = (channelVariance * largeScale).unsqueeze(-2)
prod = torch.sum(interm_var1 * interm_var2, dim=1) ** 2
diag_vec = prod.diagonal(dim1=-2, dim2=-1).unsqueeze(-1) * torch.eye(powerMatrix.shape[2], device=powerMatrix.device)

pilotContamination = torch.bmm(
    pilotAssignment,
    pilotAssignment.transpose(1, 2),
).abs()

pilotContamination = (prod - diag_vec) * pilotContamination
pilotContamination = torch.sum(pilotContamination, dim=1)


SINR_denom = 1 + userInterference + pilotContamination
rate = torch.log2(1 + SINR_num/SINR_denom)

In [ ]:
from Utils.training import rate_calculation

tmp = rate_calculation(powerMatrix, largeScale, channelVariance, pilotAssignment)

In [230]:
rate - tmp

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]], device='cuda:0')

# Others

In [ ]:
DS_all = torch.zeros((M,N_graph,K,1))
PC_all = torch.zeros((M,N_graph,K,K))
UI_all = torch.zeros((M,N_graph,K,1))

In [ ]:
from Utils.training import train, eval

records = []
for epoch in range(num_epochs):
    with torch.no_grad():
        test_min_rate = eval(test_loader, model)
        train_min_rate = eval(train_loader, model)
    train_loss = train(train_loader, model, optimizer)
    records.append([train_loss, train_min_rate, test_min_rate])
    if epoch % 10 == 1:
        print(
            f"Epoch {epoch:02d}, "
            f"Train Loss: {train_loss:.4f},"
            f"Train Min Rate: {train_min_rate:.4f},"
            f"Test Min Rate: {test_min_rate:.4f},"
        )
    scheduler.step()

AttributeError: 'GlobalStorage' object has no attribute 'x_dict'

In [15]:
# Generate Phii for num_H samples
Phii_All = np.zeros((num_H,K,tau))
for ite in range(num_H):
  Phii = np.zeros((K,tau))
  for k in range(K):
    Point = k % tau
    # Point = np.random.randint(1, tau+1)
    Phii[k,:] = U[Point - 1,:]
  Phii_All[ite,:,:] = Phii

Phii_Test = np.zeros((num_test,K,tau))
for ite in range(num_test):
  Phii = np.zeros((K,tau))
  for k in range(K):
    Point = np.random.randint(1, tau+1)
    Phii[k,:] = U[Point - 1,:]
  Phii_Test[ite,:,:] = Phii

BETAA_ALL = Generate_Input(num_H, tau, K, M, Pd, D=1, Hb=15, Hm=1.65, f=1900,
                    var_noise=1, Pmin=0, seed=2017, d0=d0, d1=d1)
BETAA_Test = Generate_Input(num_test, tau, K, M, Pd, D=1, Hb=15, Hm=1.65, f=1900,
                    var_noise=1, Pmin=0, seed=2017, d0=d0, d1=d1)
Eta_All = Generate_Label(num_H, tau, K, M, BETAA_ALL,Phii_All, Pu, Pd)
Eta_Test = Generate_Label(num_test, tau, K, M, BETAA_Test,Phii_Test, Pu, Pd)

In [16]:
train_data_list_all = proc_data(BETAA_ALL, Phii_All, Eta_All, Pd)
test_data_list_all = proc_data(BETAA_Test,Phii_Test,Eta_Test, Pd)
selectedAP = 0
train_data_list = []
test_data_list = []
for m in range(M):
  train_data_list.append([train_data_list_all[i+m] for i in range(0,num_H*M,M)]) # select the first AP to train the model
  test_data_list.append([test_data_list_all[i+m] for i in range(0,num_test*M,M)])
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(torch.cuda.is_available())

True


In [21]:
train_loader = [[] for _ in range(M)]
test_loader = [[] for _ in range(M)]
for m in range(M):
  train_loader[m] = list(DataLoader(train_data_list[m], batch_size=32, shuffle=False,num_workers=0))
  test_loader[m] = list(DataLoader(test_data_list[m], batch_size=32, shuffle=False,num_workers=0))
# for m in range(M):
#   train_loader[m] = list(train_loader[m])
#   test_loader[m] = list(test_loader[m])

In [ ]:
sr_weight = 0.2
nsu_weight = 0.8
Rthr = 3

# Load your model
# %cd '/content/drive/MyDrive/Mylearning/GNN_FL_based_Cell_Free_M_MIMO(Personal)'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = IGCNet().to(device)
# model.load_state_dict(torch.load('Decen_10Antens_all.pth'))

# model = IGCNet().to(device)
# use_gpu = torch.cuda.is_available()
# if use_gpu:
#     model = model.cuda()


optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.9)

LossSave=[]
TestLossSave=[]
for epoch in range(1,100):
  model.train()

  total_loss_train = 0
  for ite in range(len(train_loader[0])): # len(train_loader[0]) ~ number of batch
    data_trainn = []
    out_trainn = []
    for m in range(M): # 2
      data = train_loader[m][ite]
      data = data.to(device)
      data_trainn.append(data)
      if m==0:
        out_trainn.append(model(data))
      else:
        with torch.no_grad():
            out_trainn.append(model(data))

    # Loss calculation
    loss,_,_ = sr_loss_matrix(data_trainn, out_trainn, K, M, sr_weight, nsu_weight, Rthr, tau)
    loss.backward()
    total_loss_train += loss.item() * train_loader[m][ite].num_graphs
    optimizer.step()
    optimizer.zero_grad()
  loss1 = total_loss_train/num_H*-1
  LossSave.append(loss1)
  if (epoch % 1 ==0):
    model.eval()

    total_loss_test = 0
    sumrate_test = 0
    num_satisfied_test = 0
    for ite in range(len(test_loader[0])):
      data_testt = []
      out_testt = []
      for m in range(M):
        data_testt.append(test_loader[m][ite])
        out_testt.append(model(test_loader[m][ite].to(device)))
      # Loss calculation
      with torch.no_grad():
        loss,sumrate,num_satisfied = sr_loss_matrix(data_testt, out_testt, K, M, sr_weight, nsu_weight, Rthr)
        # print(f"loss test: {loss.item()}")
        total_loss_test += loss.item() * test_loader[m][ite].num_graphs
        sumrate_test += sumrate.item() * test_loader[m][ite].num_graphs
        num_satisfied_test += num_satisfied.item() * test_loader[m][ite].num_graphs
        # print(f"total_loss_test: {total_loss_test}")
    # print(num_test)
    loss2 = total_loss_test/num_test*-1
    sumrate2 = sumrate_test/num_test
    num_satisfied2 = num_satisfied_test/num_test
    TestLossSave.append(loss2)
    print('Epoch {:03d}, Train Loss: {:.4f}, Val Loss: {:.4f}, Sum rate: {:.4f}, Number of satisfied Users: {:.4f}'.format(
            epoch, loss1, loss2, sumrate2, num_satisfied2))
    scheduler.step()

/home/wine-4/miniconda3/envs/GNN_FL/lib/python3.10/site-packages/torch_geometric/warnings.py:17: UserWarning: The usage of `scatter(reduce='max')` can be accelerated via the 'torch-scatter' package, but it was not found
  warnings.warn(message)


NameError: name 'tau' is not defined